# NBA Championship Prediction Model_ v2

In [1]:
# Importing the ability to use Pandas
import pandas as pd
# Importing the ability to use Numpy and work with Arrays
import numpy as np
# Importing the ability to look at all the rows and columns of our dataset
pd.set_option('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [2]:
# Read the data from 2000 and up
nba = pd.read_excel("data/NBA_Stats_2000.xlsx")


In [3]:
nba.head()

,Rk,Team,Games Played,Minutes Played,Opp Field Goals,Opp Field Goal Attempts,Opp Field Goal Percentage,Opp 3-Pointers,Opp 3-Pointer Attempts,Opp 3-Pointer Percentage,Opp 2-Pointers,Opp 2-Pointer Attempts,Opp 2-Pointer Percentage,Opp Free-Throws,Opp Free-Throw Attempts,Opp Free-Throw Percentage,Opp Offensive Rebounds,Opp Defensive Rebounds,Opp Total Rebounds,Opp Assists,Opp Steals,Opp Blocks,Opp Turnovers,Opp Personal Fouls,Opp Points,Year,Longest Win Streak,Top 3 Conference,Accolades,Champion,Off Field Goals,Off Field Goal Attempts,Off Field Goal Percentage,Off 3-Pointers,Off 3-Pointer Attempts,Off 3-Pointer Percentage,Off 2-Pointers,Off 2-Pointer Attempts,Off 2-Pointer Percentage,Off Free-Throws,Off Free-Throw Attempts,Off Free-Throw Percentage,Off Offensive Rebounds,Off Defensive Rebounds,Off Total Rebounds,Off Assists,Off Steals,Off Blocks,Off Turnovers,Off Personal Fouls,Off Points,Division,Conference,W,L,W/L%,SRS,Mean Ht,Mean Wt,Mean Exp,HOME_percent,ROAD_percent,Pace,Net Rating,Attendance,MVP
0,1,San Antonio Spurs,82,242.134146,35.170732,82.695122,0.425000,4.329268,12.634146,0.343000,30.841463,70.060976,0.440000,15.560976,21.048780,0.739000,12.024390,29.390244,41.414634,20.329268,7.865854,5.231707,14.402439,23.085366,90.231707,2000,7,No,5,No,36.000000,77.963415,0.462000,4.024390,10.756098,0.374000,31.975610,67.207317,0.476000,20.146341,27.000000,0.746000,11.304878,32.512195,43.817073,22.182927,7.487805,6.719512,15.036585,20.926829,96.170732,Southwest,Western,53,29,0.646000,5.920000,78.333333,213.200000,7.866667,0.878000,0.488000,90.8,6.500000,884450,Yes
1,2,New York Knicks,82,241.829268,33.060976,78.024390,0.424000,4.926829,14.597561,0.338000,28.134146,63.426829,0.444000,19.621951,26.256098,0.747000,11.304878,29.768293,41.073171,19.621951,7.353659,4.317073,14.158537,21.939024,90.670732,2000,5,Yes,1,No,35.329268,77.731707,0.455000,4.280488,11.426829,0.375000,31.048780,66.304878,0.468000,17.195122,22.012195,0.781000,9.780488,30.743902,40.524390,19.365854,6.280488,4.256098,14.646341,24.182927,92.134146,Atlantic,Eastern,50,32,0.610000,1.300000,79.562500,218.500000,5.937500,0.805000,0.463000,89.2,1.600000,810103,No
2,3,Portland Trail Blazers,82,241.219512,34.451220,79.963415,0.431000,4.804878,14.573171,0.330000,29.646341,65.390244,0.453000,17.341463,24.207317,0.716000,11.926829,27.073171,39.000000,20.792683,7.951220,4.268293,14.463415,22.829268,91.048780,2000,11,Yes,1,No,36.841463,78.414634,0.470000,4.963415,13.756098,0.361000,31.878049,64.658537,0.493000,18.804878,24.743902,0.760000,11.780488,31.219512,43.000000,23.475610,7.719512,4.829268,15.158537,22.743902,97.451220,Pacific,Western,59,23,0.720000,6.360000,79.714286,218.285714,7.142857,0.805000,0.415000,89.900000,7.100000,835078,No
3,4,Miami Heat,82,241.829268,33.926829,80.426829,0.422000,4.975610,13.780488,0.361000,28.951220,66.646341,0.434000,18.439024,24.658537,0.748000,11.878049,28.548780,40.426829,19.292683,7.573171,4.439024,14.024390,21.890244,91.268293,2000,7,Yes,3,No,36.268293,78.804878,0.460000,5.439024,14.658537,0.371000,30.829268,64.146341,0.481000,16.402439,22.280488,0.736000,11.231707,31.939024,43.170732,23.548780,7.097561,6.390244,15.012195,23.743902,94.378049,Atlantic,Eastern,52,30,0.634000,2.750000,78.750000,216.500000,5.437500,0.707000,0.488000,89.700000,3.400000,706725,No
4,5,Utah Jazz,82,240.914634,33.914634,76.097561,0.446000,4.731707,13.378049,0.354000,29.182927,62.719512,0.465000,19.487805,26.109756,0.746000,10.817073,27.268293,38.085366,19.756098,7.170732,5.012195,15.487805,24.670732,92.048780,2000,9,Yes,3,No,36.121951,77.804878,0.464000,4.012195,10.414634,0.385000,32.109756,67.390244,0.476000,20.256098,26.219512,0.773000,11.414634,29.585366,41.000000,24.890244,7.670732,5.439024,14.878049,24.548780,96.512195,Northwest,Western,55,27,0.671000,4.520000,79.153846,219.000000,7.538462,0.732000,0.463000,89.600000,5,801268,Yes


In [65]:
#nba.info()

In [4]:
# Creating different variables
nba['Assist_to_Turnover_Ratio'] = nba['Off Assists'] / nba['Off Turnovers']
nba['Margin_of_Victory'] = nba['Off Points'] - nba['Opp Points']

In [5]:
# Let's get the Categorical Variables
categorical_nba = nba[['Champion', 'Top 3 Conference', 'Division', 'Conference']]
numerical_nba = nba.drop(['Champion', 'Top 3 Conference', 'Division', 'Conference', 'Team'], axis = 1)


In [6]:
# Importing the ability to One Hot Encode our data
from sklearn.preprocessing import OneHotEncoder

# Creating the One Hot Encoder
encoder = OneHotEncoder(drop = 'if_binary', sparse_output = False).set_output(transform = 'pandas')
# Encoding the Categorical Data
encoded_data = encoder.fit_transform(categorical_nba)
# Having the finalized dataset
nba_hot_encoded = numerical_nba.join(encoded_data)

In [7]:
# add warning ignore
import warnings
warnings.filterwarnings("ignore")

In [8]:

nba_hot_encoded.corr()['Champion_Yes'].sort_values(ascending = False)[:10]

Champion_Yes                1.000000
Top 3 Conference_Yes        0.371111
Longest Win Streak          0.299523
W/L%                        0.277664
Accolades                   0.276180
Margin_of_Victory           0.273716
SRS                         0.273275
W                           0.273071
Off Field Goal Percentage   0.217269
Mean Exp                    0.209728
Name: Champion_Yes, dtype: float64

In [27]:
#nba_hot_encoded

In [28]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

# Feature values from 2000 forward
feature_vector = nba_hot_encoded[['Longest Win Streak', 'Margin_of_Victory', 'Top 3 Conference_Yes', 'W/L%' ]]
# Target values form 2000 forward
target_vector = nba_hot_encoded['Champion_Yes']

# This is our training data   (seperate on 2022)
x_train = feature_vector[:655]
y_train = target_vector[:655]


# This is our testing data --> Looking at the 2021-2022 season
x_test = feature_vector[655:]
y_test = target_vector[655:]

In [29]:
# Sacling our data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

### Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model.fit(x_train_scaled, y_train)

LogisticRegression()

In [31]:
#
probabilities = log_model.predict_proba(x_test)
probabilities_df = pd.DataFrame(data = probabilities)
normalized_probabilities_df = probabilities_df.div(probabilities_df.sum(axis=0), axis=1)
normalized_probabilities_df

,0,1
0,0.007971,0.205270
1,0.036462,0.012121
2,0.017839,0.138369
3,0.027528,0.072686
4,0.037499,0.005093
5,0.038122,0.000869
6,0.037143,0.007505
7,0.002647,0.241361
8,0.037002,0.008463
9,0.027917,0.070051


In [42]:
# add to df (Logistic Regression)
nba_prac = nba[['Year', 'Team', 'Top 3 Conference', 'Margin_of_Victory', 'Longest Win Streak', 'W/L%']][655:]
nba_prac = nba_prac.reset_index()
percentages = nba_prac.join(normalized_probabilities_df).drop('index', axis = 1)
percentages = percentages.rename({0:'Probability to Lose Championship', 1:'Probability to Win Championship'}, axis = 1)
percentages


,Year,Team,Top 3 Conference,Margin_of_Victory,Longest Win Streak,W/L%,Probability to Lose Championship,Probability to Win Championship
0,2022,Boston Celtics,Yes,7.300000,9,0.622000,0.007971,0.205270
1,2022,Dallas Mavericks,No,3.300000,6,0.634000,0.036462,0.012121
2,2022,Golden State Warriors,Yes,5.500000,9,0.646000,0.017839,0.138369
3,2022,Miami Heat,Yes,4.400000,6,0.646000,0.027528,0.072686
4,2022,Cleveland Cavaliers,No,2.100000,6,0.537000,0.037499,0.005093
5,2022,New York Knicks,No,-0.100000,4,0.451000,0.038122,0.000869
6,2022,Toronto Raptors,No,2.300000,8,0.585000,0.037143,0.007505
7,2022,Phoenix Suns,Yes,7.500000,18,0.780000,0.002647,0.241361
8,2022,Philadelphia 76ers,No,2.600000,7,0.622000,0.037002,0.008463
9,2022,Utah Jazz,No,6.000000,8,0.598000,0.027917,0.070051


### Random Forest

In [43]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=0)
rf_model.fit(x_train_scaled, y_train)

# Predict probabilities for the scaled testing data
probabilities_rf = rf_model.predict_proba(x_test_scaled)

# Convert the array of probabilities to a DataFrame and normalize them
probabilities_rf_df = pd.DataFrame(data = probabilities_rf)
normalized_probabilities_rf_df = probabilities_rf_df.div(probabilities_rf_df.sum(axis=0), axis=1)
normalized_probabilities_rf_df


,0,1
0,0.027939,0.276923
1,0.034072,0.000000
2,0.033731,0.015385
3,0.034072,0.000000
4,0.034072,0.000000
5,0.034072,0.000000
6,0.034072,0.000000
7,0.029302,0.215385
8,0.034072,0.000000
9,0.028279,0.261538


In [44]:
# Add to df (RandomForest)
nba_prac = nba[['Year', 'Team', 'Top 3 Conference', 'Margin_of_Victory', 'Longest Win Streak', 'W/L%']][655:]
nba_prac = nba_prac.reset_index()
percentages_rf = nba_prac.join(normalized_probabilities_rf_df).drop('index', axis = 1)
percentages_rf = percentages_rf.rename({0:'Probability to Lose Championship', 1:'Probability to Win Championship'}, axis = 1)
percentages_rf

,Year,Team,Top 3 Conference,Margin_of_Victory,Longest Win Streak,W/L%,Probability to Lose Championship,Probability to Win Championship
0,2022,Boston Celtics,Yes,7.300000,9,0.622000,0.027939,0.276923
1,2022,Dallas Mavericks,No,3.300000,6,0.634000,0.034072,0.000000
2,2022,Golden State Warriors,Yes,5.500000,9,0.646000,0.033731,0.015385
3,2022,Miami Heat,Yes,4.400000,6,0.646000,0.034072,0.000000
4,2022,Cleveland Cavaliers,No,2.100000,6,0.537000,0.034072,0.000000
5,2022,New York Knicks,No,-0.100000,4,0.451000,0.034072,0.000000
6,2022,Toronto Raptors,No,2.300000,8,0.585000,0.034072,0.000000
7,2022,Phoenix Suns,Yes,7.500000,18,0.780000,0.029302,0.215385
8,2022,Philadelphia 76ers,No,2.600000,7,0.622000,0.034072,0.000000
9,2022,Utah Jazz,No,6.000000,8,0.598000,0.028279,0.261538


### Gradient Boosting

In [45]:
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(random_state=0)
gb_model.fit(x_train_scaled, y_train)

# Predict probabilities for the scaled testing data
probabilities_gb = gb_model.predict_proba(x_test_scaled)

# Convert the array of probabilities to a DataFrame and normalize them
probabilities_gb_df = pd.DataFrame(data = probabilities_gb)
normalized_probabilities_gb_df = probabilities_gb_df.div(probabilities_gb_df.sum(axis=0), axis=1)
normalized_probabilities_gb_df

,0,1
0,0.033209,0.088946
1,0.033297,0.049426
2,0.033198,0.093956
3,0.033283,0.055780
4,0.033393,0.006777
5,0.033393,0.006777
6,0.033383,0.011084
7,0.033038,0.165489
8,0.033393,0.006777
9,0.033313,0.042226


In [46]:
# add to df (Gradient Boosting)
nba_prac = nba[['Year', 'Team', 'Top 3 Conference', 'Margin_of_Victory', 'Longest Win Streak', 'W/L%']][655:]
nba_prac = nba_prac.reset_index()
percentages_gb = nba_prac.join(normalized_probabilities_gb_df).drop('index', axis = 1)
percentages_gb = percentages_gb.rename({0:'Probability to Lose Championship', 1:'Probability to Win Championship'}, axis = 1)
percentages_gb

,Year,Team,Top 3 Conference,Margin_of_Victory,Longest Win Streak,W/L%,Probability to Lose Championship,Probability to Win Championship
0,2022,Boston Celtics,Yes,7.300000,9,0.622000,0.033209,0.088946
1,2022,Dallas Mavericks,No,3.300000,6,0.634000,0.033297,0.049426
2,2022,Golden State Warriors,Yes,5.500000,9,0.646000,0.033198,0.093956
3,2022,Miami Heat,Yes,4.400000,6,0.646000,0.033283,0.055780
4,2022,Cleveland Cavaliers,No,2.100000,6,0.537000,0.033393,0.006777
5,2022,New York Knicks,No,-0.100000,4,0.451000,0.033393,0.006777
6,2022,Toronto Raptors,No,2.300000,8,0.585000,0.033383,0.011084
7,2022,Phoenix Suns,Yes,7.500000,18,0.780000,0.033038,0.165489
8,2022,Philadelphia 76ers,No,2.600000,7,0.622000,0.033393,0.006777
9,2022,Utah Jazz,No,6.000000,8,0.598000,0.033313,0.042226


### XGBoost 

In [47]:
from xgboost import XGBClassifier


xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(x_train_scaled, y_train)

# Predict probabilities for the scaled testing data
probabilities_xgb = xgb_model.predict_proba(x_test_scaled)

# Convert the array of probabilities to a DataFrame and normalize them
probabilities_xgb_df = pd.DataFrame(data = probabilities_xgb)
normalized_probabilities_xgb_df = probabilities_xgb_df.div(probabilities_xgb_df.sum(axis=0), axis=1)
normalized_probabilities_xgb_df

,0,1
0,0.032192,0.324275
1,0.033434,0.007673
2,0.033435,0.007485
3,0.033461,0.000757
4,0.033452,0.003112
5,0.033452,0.003112
6,0.033440,0.006212
7,0.033141,0.082347
8,0.033452,0.003112
9,0.033116,0.088719


In [48]:
# add to df (XGBoost)
nba_prac = nba[['Year', 'Team', 'Top 3 Conference', 'Margin_of_Victory', 'Longest Win Streak', 'W/L%']][655:]
nba_prac = nba_prac.reset_index()
percentages_xgb = nba_prac.join(normalized_probabilities_xgb_df).drop('index', axis = 1)
percentages_xgb = percentages_gb.rename({0:'Probability to Lose Championship', 1:'Probability to Win Championship'}, axis = 1)
percentages_xgb

,Year,Team,Top 3 Conference,Margin_of_Victory,Longest Win Streak,W/L%,Probability to Lose Championship,Probability to Win Championship
0,2022,Boston Celtics,Yes,7.300000,9,0.622000,0.033209,0.088946
1,2022,Dallas Mavericks,No,3.300000,6,0.634000,0.033297,0.049426
2,2022,Golden State Warriors,Yes,5.500000,9,0.646000,0.033198,0.093956
3,2022,Miami Heat,Yes,4.400000,6,0.646000,0.033283,0.055780
4,2022,Cleveland Cavaliers,No,2.100000,6,0.537000,0.033393,0.006777
5,2022,New York Knicks,No,-0.100000,4,0.451000,0.033393,0.006777
6,2022,Toronto Raptors,No,2.300000,8,0.585000,0.033383,0.011084
7,2022,Phoenix Suns,Yes,7.500000,18,0.780000,0.033038,0.165489
8,2022,Philadelphia 76ers,No,2.600000,7,0.622000,0.033393,0.006777
9,2022,Utah Jazz,No,6.000000,8,0.598000,0.033313,0.042226


### Support Vector Machine (SVM)

In [49]:
from sklearn.svm import SVC


svm_model = SVC(probability=True, random_state=0)
svm_model.fit(x_train_scaled, y_train)

# Predict probabilities for the scaled testing data
probabilities_svm = svm_model.predict_proba(x_test_scaled)

# Convert the array of probabilities to a DataFrame and normalize them
probabilities_svm_df = pd.DataFrame(data = probabilities_svm)
normalized_probabilities_svm_df = probabilities_svm_df.div(probabilities_svm_df.sum(axis=0), axis=1)
normalized_probabilities_svm_df

,0,1
0,0.033322,0.033651
1,0.033334,0.033311
2,0.033326,0.033534
3,0.033328,0.033493
4,0.033337,0.033232
5,0.033336,0.033250
6,0.033338,0.033215
7,0.033337,0.033226
8,0.033335,0.033274
9,0.033330,0.033439


In [50]:
# add to df (SVM)
nba_prac = nba[['Year', 'Team', 'Top 3 Conference', 'Margin_of_Victory', 'Longest Win Streak', 'W/L%']][655:]
nba_prac = nba_prac.reset_index()
percentages_svm= nba_prac.join(normalized_probabilities_svm_df).drop('index', axis = 1)
percentages_svm= percentages_gb.rename({0:'Probability to Lose Championship', 1:'Probability to Win Championship'}, axis = 1)
percentages_svm

,Year,Team,Top 3 Conference,Margin_of_Victory,Longest Win Streak,W/L%,Probability to Lose Championship,Probability to Win Championship
0,2022,Boston Celtics,Yes,7.300000,9,0.622000,0.033209,0.088946
1,2022,Dallas Mavericks,No,3.300000,6,0.634000,0.033297,0.049426
2,2022,Golden State Warriors,Yes,5.500000,9,0.646000,0.033198,0.093956
3,2022,Miami Heat,Yes,4.400000,6,0.646000,0.033283,0.055780
4,2022,Cleveland Cavaliers,No,2.100000,6,0.537000,0.033393,0.006777
5,2022,New York Knicks,No,-0.100000,4,0.451000,0.033393,0.006777
6,2022,Toronto Raptors,No,2.300000,8,0.585000,0.033383,0.011084
7,2022,Phoenix Suns,Yes,7.500000,18,0.780000,0.033038,0.165489
8,2022,Philadelphia 76ers,No,2.600000,7,0.622000,0.033393,0.006777
9,2022,Utah Jazz,No,6.000000,8,0.598000,0.033313,0.042226


### Decision Tree 

In [51]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=0)
dt_model.fit(x_train_scaled, y_train)

# Predict probabilities for the scaled testing data
probabilities_dt = dt_model.predict_proba(x_test_scaled)

# Convert the array of probabilities to a DataFrame and normalize them
probabilities_dt_df = pd.DataFrame(data = probabilities_dt)
normalized_probabilities_dt_df = probabilities_dt_df.div(probabilities_dt_df.sum(axis=1), axis=0)
normalized_probabilities_dt_df

,0,1
0,1.000000,0.000000
1,1.000000,0.000000
2,1.000000,0.000000
3,1.000000,0.000000
4,1.000000,0.000000
5,1.000000,0.000000
6,1.000000,0.000000
7,1.000000,0.000000
8,1.000000,0.000000
9,1.000000,0.000000


## Accuracy

In [52]:
from sklearn.metrics import accuracy_score

# Predict the labels for the test set
y_pred_log = log_model.predict(x_test_scaled)
y_pred_rf = rf_model.predict(x_test_scaled)
y_pred_xgb = xgb_model.predict(x_test_scaled)
y_pred_svm = svm_model.predict(x_test_scaled)
y_pred_dt = dt_model.predict(x_test_scaled) 

# Calculate the accuracy of each model
accuracy_log = accuracy_score(y_test, y_pred_log)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
accuracy_dt = accuracy_score(y_test, y_pred_dt)

# Print the accuracy of each model
print("Accuracy of Logistic Regression: ", accuracy_log)
print("Accuracy of Random Forest: ", accuracy_rf)
print("Accuracy of XGBoost: ", accuracy_xgb)
print("Accuracy of SVM: ", accuracy_svm)
print("Accuracy of Decision Tree: ", accuracy_dt)


Accuracy of Logistic Regression:  0.9666666666666667
Accuracy of Random Forest:  0.9666666666666667
Accuracy of XGBoost:  0.9666666666666667
Accuracy of SVM:  0.9666666666666667
Accuracy of Decision Tree:  0.9666666666666667


## Compare all models

In [53]:
# Select only the "win" column from each DataFrame
win_probabilities_lr = normalized_probabilities_df[1]
win_probabilities_rf = normalized_probabilities_rf_df[1]
win_probabilities_xgb = normalized_probabilities_xgb_df[1]
win_probabilities_svm = normalized_probabilities_svm_df[1]
win_probabilities_dt = normalized_probabilities_dt_df[1]

# Concatenate all the "win" probability series along the column axis
all_probabilities_df = pd.concat([win_probabilities_lr, win_probabilities_rf, win_probabilities_xgb, 
                                  win_probabilities_svm, win_probabilities_dt], axis=1)

# Rename the columns
all_probabilities_df.columns = ['Logistic Regression',
                                'Random Forest', 
                                'XGBoost', 
                                'SVM',
                                'Decision Tree']


all_probabilities_df

,Logistic Regression,Random Forest,XGBoost,SVM,Decision Tree
0,0.205270,0.276923,0.324275,0.033651,0.000000
1,0.012121,0.000000,0.007673,0.033311,0.000000
2,0.138369,0.015385,0.007485,0.033534,0.000000
3,0.072686,0.000000,0.000757,0.033493,0.000000
4,0.005093,0.000000,0.003112,0.033232,0.000000
5,0.000869,0.000000,0.003112,0.033250,0.000000
6,0.007505,0.000000,0.006212,0.033215,0.000000
7,0.241361,0.215385,0.082347,0.033226,0.000000
8,0.008463,0.000000,0.003112,0.033274,0.000000
9,0.070051,0.261538,0.088719,0.033439,0.000000


In [54]:
# Reset the index of all_probabilities_df
all_probabilities_df = all_probabilities_df.reset_index(drop=True)

# Add to df
nba_prac = nba[['Year', 'Team', 'Top 3 Conference', 'Margin_of_Victory', 'Longest Win Streak', 'W/L%']][655:]
nba_prac = nba_prac.reset_index(drop=True)
percentages_all = nba_prac.join(all_probabilities_df)

# Display the dataframe
percentages_all

,Year,Team,Top 3 Conference,Margin_of_Victory,Longest Win Streak,W/L%,Logistic Regression,Random Forest,XGBoost,SVM,Decision Tree
0,2022,Boston Celtics,Yes,7.300000,9,0.622000,0.205270,0.276923,0.324275,0.033651,0.000000
1,2022,Dallas Mavericks,No,3.300000,6,0.634000,0.012121,0.000000,0.007673,0.033311,0.000000
2,2022,Golden State Warriors,Yes,5.500000,9,0.646000,0.138369,0.015385,0.007485,0.033534,0.000000
3,2022,Miami Heat,Yes,4.400000,6,0.646000,0.072686,0.000000,0.000757,0.033493,0.000000
4,2022,Cleveland Cavaliers,No,2.100000,6,0.537000,0.005093,0.000000,0.003112,0.033232,0.000000
5,2022,New York Knicks,No,-0.100000,4,0.451000,0.000869,0.000000,0.003112,0.033250,0.000000
6,2022,Toronto Raptors,No,2.300000,8,0.585000,0.007505,0.000000,0.006212,0.033215,0.000000
7,2022,Phoenix Suns,Yes,7.500000,18,0.780000,0.241361,0.215385,0.082347,0.033226,0.000000
8,2022,Philadelphia 76ers,No,2.600000,7,0.622000,0.008463,0.000000,0.003112,0.033274,0.000000
9,2022,Utah Jazz,No,6.000000,8,0.598000,0.070051,0.261538,0.088719,0.033439,0.000000
